In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import tensorflow


from sklearn.model_selection import train_test_split, GridSearchCV


import sys
sys.path.append('../')
from scripts import featurizer_rn


%load_ext autoreload
%autoreload 2

2022-08-10 12:37:07.397207: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-08-10 12:37:07.442474: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-10 12:37:07.442482: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df = pd.read_csv("../../data/dataset-procesado.csv", low_memory=False)

In [3]:
df.head()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,D_43,D_44,B_4,...,R_28,D_139,D_140,D_144,D_145,customer_ID,S_2,D_63,D_64,target
0,0.947242,0.002204,0.008002,1.007516,0.002830,0.453227,0.005992,0.153998,0.001613,0.029088,...,0.000518,0.003183,0.005264,0.004306,0.002719,743d55408505dee752adc6e62b273f4397db6e89f19034...,2017-08-10,CO,O,0
1,0.757648,0.002076,0.008751,0.814083,0.000408,0.155700,0.004156,0.127285,0.129051,0.040101,...,0.001186,1.000928,0.005655,0.007953,0.095926,522c85a354bfec790067b54e9db7353d20d4709ae5dc93...,2017-07-04,CO,O,0
2,0.673978,0.588604,0.295201,0.024650,0.007951,0.125583,0.008760,0.153998,0.005867,0.175950,...,0.000805,0.003558,0.008268,0.003020,0.002798,9296e983c3e4f99b046952397fc4ba950abf1032a9bd6c...,2017-03-30,CO,R,1
3,0.778075,0.006930,0.056721,0.819369,0.004922,0.089214,0.006884,0.041515,0.002841,0.001179,...,0.009053,0.004670,0.000125,0.000819,0.000342,3fa10314686e0e51d8f76235eceab91d70aa3f24a98c3e...,2018-01-17,CL,O,0
4,0.561622,0.032756,0.822028,0.025315,0.005570,1.147400,0.008290,0.153998,0.002991,0.316650,...,0.008689,0.003884,0.003367,0.009010,0.004627,1f81653f1397fe11839b585cb688e2e92ce1805023ba67...,2017-06-21,CO,O,1


In [4]:
df.shape

(263965, 123)

In [5]:
X, Y = featurizer_rn.featurizer(df)

In [6]:
X.shape

(263965, 148)

In [7]:
dataset = X.values

In [8]:
Y

0         0
1         0
2         1
3         0
4         1
         ..
263960    0
263961    1
263962    0
263963    0
263964    1
Name: target, Length: 263965, dtype: int64

In [9]:
X = dataset.astype(float)
y = Y

In [10]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)

In [11]:
for train, test in kfold.split(X, y):
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]

In [12]:
X.shape[0]

263965

In [13]:
X_train.shape[0]/X.shape[0]

0.8

In [14]:
X_test.shape[0]/X.shape[0]

0.2

In [15]:
# baseline model
def create_baseline(input_dim, opt):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=input_dim, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation="sigmoid"))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['binary_accuracy'])
    return model

In [16]:
from keras.optimizers import SGD
opt = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model_baseline = create_baseline(X_train.shape[1],opt)

2022-08-10 12:37:33.508023: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-10 12:37:33.508046: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-10 12:37:33.508064: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (latitude5421): /proc/driver/nvidia/version does not exist
2022-08-10 12:37:33.508465: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
model_baseline.fit(X_train, y_train, epochs=1000, verbose=1, batch_size=100,validation_split=0.2)

Epoch 1/1000
1690/1690 [==============================] - 1s 645us/step - loss: 0.3445 - binary_accuracy: 0.8401 - val_loss: 0.3003 - val_binary_accuracy: 0.8626
Epoch 2/1000
1690/1690 [==============================] - 1s 574us/step - loss: 0.2975 - binary_accuracy: 0.8633 - val_loss: 0.2980 - val_binary_accuracy: 0.8611
Epoch 3/1000
1690/1690 [==============================] - 1s 584us/step - loss: 0.2930 - binary_accuracy: 0.8643 - val_loss: 0.3030 - val_binary_accuracy: 0.8583
Epoch 4/1000
1690/1690 [==============================] - 1s 577us/step - loss: 0.2911 - binary_accuracy: 0.8660 - val_loss: 0.2898 - val_binary_accuracy: 0.8648
Epoch 5/1000
1690/1690 [==============================] - 1s 600us/step - loss: 0.2896 - binary_accuracy: 0.8663 - val_loss: 0.2891 - val_binary_accuracy: 0.8658
Epoch 6/1000
1690/1690 [==============================] - 1s 590us/step - loss: 0.2887 - binary_accuracy: 0.8671 - val_loss: 0.3029 - val_binary_accuracy: 0.8613
Epoch 7/1000
1690/1690 [====

1690/1690 [==============================] - 1s 682us/step - loss: 0.2810 - binary_accuracy: 0.8710 - val_loss: 0.2864 - val_binary_accuracy: 0.8676
Epoch 52/1000
1690/1690 [==============================] - 1s 667us/step - loss: 0.2809 - binary_accuracy: 0.8710 - val_loss: 0.2869 - val_binary_accuracy: 0.8683
Epoch 53/1000
1690/1690 [==============================] - 1s 692us/step - loss: 0.2807 - binary_accuracy: 0.8711 - val_loss: 0.2821 - val_binary_accuracy: 0.8700
Epoch 54/1000
1690/1690 [==============================] - 1s 690us/step - loss: 0.2807 - binary_accuracy: 0.8711 - val_loss: 0.2834 - val_binary_accuracy: 0.8698
Epoch 55/1000
1690/1690 [==============================] - 1s 685us/step - loss: 0.2805 - binary_accuracy: 0.8708 - val_loss: 0.2828 - val_binary_accuracy: 0.8696
Epoch 56/1000
1690/1690 [==============================] - 1s 689us/step - loss: 0.2805 - binary_accuracy: 0.8714 - val_loss: 0.2969 - val_binary_accuracy: 0.8634
Epoch 57/1000
1690/1690 [===========

1690/1690 [==============================] - 1s 685us/step - loss: 0.2793 - binary_accuracy: 0.8721 - val_loss: 0.2824 - val_binary_accuracy: 0.8688
Epoch 102/1000
1690/1690 [==============================] - 1s 688us/step - loss: 0.2793 - binary_accuracy: 0.8718 - val_loss: 0.2820 - val_binary_accuracy: 0.8696
Epoch 103/1000
1690/1690 [==============================] - 1s 682us/step - loss: 0.2790 - binary_accuracy: 0.8723 - val_loss: 0.2823 - val_binary_accuracy: 0.8706
Epoch 104/1000
1690/1690 [==============================] - 1s 683us/step - loss: 0.2792 - binary_accuracy: 0.8726 - val_loss: 0.2819 - val_binary_accuracy: 0.8703
Epoch 105/1000
1690/1690 [==============================] - 1s 681us/step - loss: 0.2790 - binary_accuracy: 0.8720 - val_loss: 0.2835 - val_binary_accuracy: 0.8704
Epoch 106/1000
1690/1690 [==============================] - 1s 681us/step - loss: 0.2790 - binary_accuracy: 0.8725 - val_loss: 0.2859 - val_binary_accuracy: 0.8699
Epoch 107/1000
1690/1690 [=====

1690/1690 [==============================] - 1s 694us/step - loss: 0.2781 - binary_accuracy: 0.8724 - val_loss: 0.2850 - val_binary_accuracy: 0.8679
Epoch 152/1000
1690/1690 [==============================] - 1s 678us/step - loss: 0.2783 - binary_accuracy: 0.8726 - val_loss: 0.2810 - val_binary_accuracy: 0.8707
Epoch 153/1000
1690/1690 [==============================] - 1s 697us/step - loss: 0.2783 - binary_accuracy: 0.8722 - val_loss: 0.2865 - val_binary_accuracy: 0.8656
Epoch 154/1000
1690/1690 [==============================] - 1s 693us/step - loss: 0.2781 - binary_accuracy: 0.8723 - val_loss: 0.2834 - val_binary_accuracy: 0.8708
Epoch 155/1000
1690/1690 [==============================] - 1s 686us/step - loss: 0.2782 - binary_accuracy: 0.8718 - val_loss: 0.2882 - val_binary_accuracy: 0.8689
Epoch 156/1000
1690/1690 [==============================] - 1s 676us/step - loss: 0.2783 - binary_accuracy: 0.8721 - val_loss: 0.2814 - val_binary_accuracy: 0.8710
Epoch 157/1000
1690/1690 [=====

1690/1690 [==============================] - 1s 673us/step - loss: 0.2775 - binary_accuracy: 0.8726 - val_loss: 0.2815 - val_binary_accuracy: 0.8706
Epoch 202/1000
1690/1690 [==============================] - 1s 671us/step - loss: 0.2776 - binary_accuracy: 0.8729 - val_loss: 0.2810 - val_binary_accuracy: 0.8698
Epoch 203/1000
1690/1690 [==============================] - 1s 685us/step - loss: 0.2775 - binary_accuracy: 0.8728 - val_loss: 0.2827 - val_binary_accuracy: 0.8699
Epoch 204/1000
1690/1690 [==============================] - 1s 666us/step - loss: 0.2776 - binary_accuracy: 0.8725 - val_loss: 0.2808 - val_binary_accuracy: 0.8712
Epoch 205/1000
1690/1690 [==============================] - 1s 679us/step - loss: 0.2776 - binary_accuracy: 0.8724 - val_loss: 0.2852 - val_binary_accuracy: 0.8678
Epoch 206/1000
1690/1690 [==============================] - 1s 674us/step - loss: 0.2776 - binary_accuracy: 0.8727 - val_loss: 0.2813 - val_binary_accuracy: 0.8706
Epoch 207/1000
1690/1690 [=====

...

1690/1690 [==============================] - 1s 671us/step - loss: 0.2749 - binary_accuracy: 0.8739 - val_loss: 0.2791 - val_binary_accuracy: 0.8721
Epoch 752/1000
1690/1690 [==============================] - 1s 682us/step - loss: 0.2750 - binary_accuracy: 0.8739 - val_loss: 0.2827 - val_binary_accuracy: 0.8703
Epoch 753/1000
1690/1690 [==============================] - 1s 670us/step - loss: 0.2749 - binary_accuracy: 0.8742 - val_loss: 0.2788 - val_binary_accuracy: 0.8726
Epoch 754/1000
1690/1690 [==============================] - 1s 676us/step - loss: 0.2750 - binary_accuracy: 0.8739 - val_loss: 0.2793 - val_binary_accuracy: 0.8729
Epoch 755/1000
1690/1690 [==============================] - 1s 686us/step - loss: 0.2749 - binary_accuracy: 0.8740 - val_loss: 0.2802 - val_binary_accuracy: 0.8711
Epoch 756/1000
1690/1690 [==============================] - 1s 687us/step - loss: 0.2749 - binary_accuracy: 0.8737 - val_loss: 0.2834 - val_binary_accuracy: 0.8688
Epoch 757/1000
1690/1690 [=====

1690/1690 [==============================] - 1s 673us/step - loss: 0.2748 - binary_accuracy: 0.8743 - val_loss: 0.2814 - val_binary_accuracy: 0.8699
Epoch 802/1000
1690/1690 [==============================] - 1s 663us/step - loss: 0.2746 - binary_accuracy: 0.8740 - val_loss: 0.2802 - val_binary_accuracy: 0.8711
Epoch 803/1000
1690/1690 [==============================] - 1s 665us/step - loss: 0.2748 - binary_accuracy: 0.8737 - val_loss: 0.2787 - val_binary_accuracy: 0.8732
Epoch 804/1000
1690/1690 [==============================] - 1s 671us/step - loss: 0.2748 - binary_accuracy: 0.8739 - val_loss: 0.2857 - val_binary_accuracy: 0.8696
Epoch 805/1000
1690/1690 [==============================] - 1s 695us/step - loss: 0.2748 - binary_accuracy: 0.8740 - val_loss: 0.2799 - val_binary_accuracy: 0.8709
Epoch 806/1000
1690/1690 [==============================] - 1s 674us/step - loss: 0.2748 - binary_accuracy: 0.8739 - val_loss: 0.2797 - val_binary_accuracy: 0.8718
Epoch 807/1000
1690/1690 [=====

1690/1690 [==============================] - 1s 677us/step - loss: 0.2749 - binary_accuracy: 0.8739 - val_loss: 0.2791 - val_binary_accuracy: 0.8723
Epoch 852/1000
1690/1690 [==============================] - 1s 670us/step - loss: 0.2747 - binary_accuracy: 0.8740 - val_loss: 0.2794 - val_binary_accuracy: 0.8718
Epoch 853/1000
1690/1690 [==============================] - 1s 663us/step - loss: 0.2745 - binary_accuracy: 0.8744 - val_loss: 0.2827 - val_binary_accuracy: 0.8690
Epoch 854/1000
1690/1690 [==============================] - 1s 667us/step - loss: 0.2746 - binary_accuracy: 0.8736 - val_loss: 0.2789 - val_binary_accuracy: 0.8718
Epoch 855/1000
1690/1690 [==============================] - 1s 691us/step - loss: 0.2747 - binary_accuracy: 0.8741 - val_loss: 0.2797 - val_binary_accuracy: 0.8708
Epoch 856/1000
1690/1690 [==============================] - 1s 686us/step - loss: 0.2747 - binary_accuracy: 0.8744 - val_loss: 0.2801 - val_binary_accuracy: 0.8728
Epoch 857/1000
1690/1690 [=====

1690/1690 [==============================] - 1s 659us/step - loss: 0.2745 - binary_accuracy: 0.8744 - val_loss: 0.2786 - val_binary_accuracy: 0.8725
Epoch 902/1000
1690/1690 [==============================] - 1s 681us/step - loss: 0.2745 - binary_accuracy: 0.8741 - val_loss: 0.2789 - val_binary_accuracy: 0.8727
Epoch 903/1000
1690/1690 [==============================] - 1s 670us/step - loss: 0.2746 - binary_accuracy: 0.8738 - val_loss: 0.2787 - val_binary_accuracy: 0.8721
Epoch 904/1000
1690/1690 [==============================] - 1s 692us/step - loss: 0.2746 - binary_accuracy: 0.8740 - val_loss: 0.2793 - val_binary_accuracy: 0.8715
Epoch 905/1000
1690/1690 [==============================] - 1s 663us/step - loss: 0.2746 - binary_accuracy: 0.8741 - val_loss: 0.2795 - val_binary_accuracy: 0.8726
Epoch 906/1000
1690/1690 [==============================] - 1s 686us/step - loss: 0.2744 - binary_accuracy: 0.8741 - val_loss: 0.2803 - val_binary_accuracy: 0.8723
Epoch 907/1000
1690/1690 [=====

1690/1690 [==============================] - 1s 691us/step - loss: 0.2744 - binary_accuracy: 0.8746 - val_loss: 0.2794 - val_binary_accuracy: 0.8726
Epoch 952/1000
1690/1690 [==============================] - 1s 686us/step - loss: 0.2745 - binary_accuracy: 0.8739 - val_loss: 0.2788 - val_binary_accuracy: 0.8717
Epoch 953/1000
1690/1690 [==============================] - 1s 680us/step - loss: 0.2745 - binary_accuracy: 0.8741 - val_loss: 0.2796 - val_binary_accuracy: 0.8728
Epoch 954/1000
1690/1690 [==============================] - 1s 677us/step - loss: 0.2745 - binary_accuracy: 0.8742 - val_loss: 0.2791 - val_binary_accuracy: 0.8728
Epoch 955/1000
1690/1690 [==============================] - 1s 679us/step - loss: 0.2745 - binary_accuracy: 0.8743 - val_loss: 0.2804 - val_binary_accuracy: 0.8720
Epoch 956/1000
1690/1690 [==============================] - 1s 684us/step - loss: 0.2746 - binary_accuracy: 0.8740 - val_loss: 0.2789 - val_binary_accuracy: 0.8728
Epoch 957/1000
1690/1690 [=====

In [18]:
scores_train = model_baseline.evaluate(X_train, y_train)
print("\n%s: %.2f%%" % (model_baseline.metrics_names[1], scores_train[1]*100))

6600/6600 [==============================] - 3s 455us/step - loss: 0.2744 - binary_accuracy: 0.8744

binary_accuracy: 87.44%


In [19]:
scores_test = model_baseline.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model_baseline.metrics_names[1], scores_test[1]*100))

1650/1650 [==============================] - 1s 442us/step - loss: 0.2767 - binary_accuracy: 0.8717

binary_accuracy: 87.17%
